# BitNet

[BitNet](https://arxiv.org/abs/2402.17764) 是一种新型的神经网络架构，它用特殊的 BitLinear 层替换了传统多头注意力机制和前馈网络中的线性层。BitLinear 层使用三值（或二值在旧版本中）精度量化权重，并将激活值量化为 8 位精度。

![BitLinear 层的 BitNet 架构](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/1.58llm_extreme_quantization/bitlinear.png)

### 训练过程

在训练过程中，我们首先使用对称张量量化方法将权重量化为三值（-1, 0, 1）。具体步骤如下：

1. **计算权重的平均绝对值**：
   $$
   \text{scale}_w = \frac{1}{\frac{1}{nm} \sum_{ij} |W_{ij}|}
   $$

   其中 $W$ 是权重矩阵， $n$ 和 $m$ 分别是矩阵的行数和列数。

2. **量化权重**：
   $$
   W_q = \text{clamp}_{[-1,1]}(\text{round}(W \times \text{scale}))
   $$
   这里，我们先将权重乘以缩放因子，然后四舍五入并限制在 -1 到 1 之间。

3. **反量化权重**：
   $$
   W_{\text{dequantized}} = W_q \times \text{scale}_w
   $$

接下来，我们将激活值量化为指定的位宽（例如，8 位），使用 [absmax](https://arxiv.org/pdf/2208.07339) 量化方法（对称通道量化）。具体步骤如下：

1. **计算激活值的最大绝对值**：
   $$
   \text{scale}_x = \frac{127}{|X|_{\text{max}, \, \text{dim}=-1}}
   $$

2. **量化激活值**：
   $$
   X_q = \text{clamp}_{[-128,127]}(\text{round}(X \times \text{scale}))
   $$

3. **反量化激活值**：
   $$
   X_{\text{dequantized}} = X_q \times \text{scale}_x
   $$

更多关于如何训练和微调 BitNet 模型的信息，请查看这篇 [博客文章](https://huggingface.co/blog/1_58_llm_extreme_quantization)。

### 从 Hub 加载 BitNet 模型

BitNet 模型不能动态量化，它们需要预先训练或微调，应用量化技术（即量化感知训练）。训练完成后，这些模型已经量化完成，并以打包的形式在 Hub 上提供。

加载量化模型的方法如下：


In [ ]:
from transformers import AutoModelForCausalLM
path = "/path/to/model"
model = AutoModelForCausalLM.from_pretrained(path, device_map="auto")


### 预训练和微调 BitNet 模型

如果你希望使用 Nanotron 预训练或微调自己的 1.58 位模型，可以参考这个 [PR](https://github.com/huggingface/nanotron/pull/180)，所有需要的信息都在那里！

对于微调，你需要将模型从 Hugging Face 格式转换为 Nanotron 格式（两者有一些差异）。转换步骤可以在这个 [PR](https://github.com/huggingface/nanotron/pull/174) 中找到。

### 内核

在我们的初始版本中，我们选择使用 `@torch.compile` 来解包权重并执行前向传播。这种方法实现非常简单，并且带来了显著的速度提升。我们计划在未来的版本中集成更多的优化内核。